# 8. Chapter: Building Trust and Transparency in LLMs

In [1]:
import requests
from datetime import datetime, timedelta
from typing import Any
from pydantic import BaseModel, Field
from language_models.agent import (
    Agent,
    OutputType,
    PromptingStrategy,
    Workflow,
    WorkflowAgentStep,
    WorkflowFunctionStep,
    WorkflowTransformationStep,
)
from language_models.tools import Tool
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

## Governed Tool Access

In [4]:
def current_date() -> datetime:
    return datetime.now()

current_date_tool = Tool(
    function=current_date,
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

class USGeopoliticalSurveyEarthquakeAPI(BaseModel):
    """Class that implements the API interface."""

    start_time: str = Field(
        None,
        description=(
            "Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    end_time: str = Field(
        None,
        description=(
            "Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    limit: int = Field(
        20000,
        description=(
            "Limit the results to the specified number of events. NOTE: The service limits queries to 20000,"
            + " and any that exceed this limit will generate a HTTP response code 400 Bad Request."
        ),
    )
    min_depth: int = Field(
        -100,
        description="Limit to events with depth more than the specified minimum.",
    )
    max_depth: int = Field(
        1000,
        description="Limit to events with depth less than the specified maximum.",
    )
    min_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude larger than the specified minimum.",
    )
    max_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude smaller than the specified maximum.",
    )
    alert_level: str = Field(
        None,
        description=(
            "Limit to events with a specific PAGER alert level."
            + " The allowed values are: alert_level=green Limit to events with PAGER"
            + ' alert level "green". alert_level=yellow Limit to events with PAGER alert level "yellow".'
            + ' alert_level=orange Limit to events with PAGER alert level "orange".'
            + ' alert_level=red Limit to events with PAGER alert level "red".'
        ),
    )

def get_earthquakes(
    endpoint: str,
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    params = {
        "format": "geojson",
        "starttime": start_time,
        "endtime": end_time,
        "limit": limit,
        "mindepth": min_depth,
        "maxdepth": max_depth,
        "minmagnitude": min_magnitude,
        "maxmagnitude": max_magnitude,
        "alertlevel": alert_level,
        "eventtype": "earthquake",
    }
    response = requests.get(
        f"https://earthquake.usgs.gov/fdsnws/event/1/{endpoint}",
        params=params,
        timeout=None,
    )
    return response.json()

def query_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="query",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

def count_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="count",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
    requires_approval=True,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
    requires_approval=True,
)

In [5]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date_tool, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=False,
)

In [6]:
count = agent.invoke({"question": "How many earthquakes occurred today?"})

In [7]:
query = agent.invoke({"question": "Show me 3."})

## Displaying the Chain-of-Thought

In [8]:
print(count.final_answer)

There were 116 earthquakes today.


In [10]:
for step in output.chain_of_thought:
    if step.name == "tool":
        print(f"{step.name}: {step.content}")
    else:
        print(f"{step.name}: {step.content}")
    print()

prompt: How many earthquakes occurred today?

thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool to count the number of earthquakes that occurred today.

tool: tool='Current Date' tool_input={} tool_output=datetime.datetime(2024, 8, 8, 10, 58, 7, 470561)

thought: Now that I have the current date, I can use the "Count Earthquakes" tool to count the number of earthquakes that occurred today. I will use the start time as the beginning of today and the end time as the current time.

tool: tool='Count Earthquakes' tool_input={'start_time': '2024-08-08T00:00:00', 'end_time': '2024-08-08T10:58:07'} tool_output={'count': 115, 'maxAllowed': 20000}

thought: The "Count Earthquakes" tool has provided the number of earthquakes that occurred today.

final_answer: There were 115 earthquakes today.



## Monitoring